In [1]:
import sys
sys.path.append('/lmh_data/work/sclab/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=False)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), 741
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(ASC=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'ASC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
#         target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        target = model(input)
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/AD/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/AD/tmp/GPM6A/model_epoch_12.pth', 'GPM6A')
important_genes

{'ASC': {0: -0.05901884311970207,
  1: -0.07864869013428688,
  2: 0.0,
  3: -0.06266803480684757,
  4: 0.0,
  5: -1.1689526177942753,
  6: 0.13113843090832233,
  7: -0.08186634443700314,
  8: 0.0,
  9: 0.025237999157980084,
  10: 2.9339311784133315,
  11: -0.4356287522241473,
  12: -0.09765889057598542,
  13: 0.0001529429864604026,
  14: 0.01599444937892258,
  15: -0.000573288751184009,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: -0.5547126960009336,
  20: -2.0786751210689545,
  21: 0.0,
  22: 0.26665535382926464,
  23: 0.0,
  24: 0.0,
  25: 0.0011192045640200377,
  26: -0.011452041920165357,
  27: 0.011674584355205297,
  28: 0.0,
  29: 0.0,
  30: -2.4185942597687244,
  31: -0.45886671356856823,
  32: 0.0,
  33: 0.27794941514730453,
  34: 0.0,
  35: -0.15264989086426795,
  36: 0.07349156867712736,
  37: -0.007405932992696762,
  38: 0.0,
  39: -3.188905426301062,
  40: 0.0,
  41: 0.0,
  42: 0.2887575887143612,
  43: -0.3957029301673174,
  44: -0.3853332732687704,
  45: 0.0,
  46: -0.43961256

In [4]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/AD/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'ASC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_GPM6A.csv')
genes

,value,name
index,,
16466,144.938489,ZBTB20
11179,124.819023,NRG3
3904,120.400032,SLC1A2
3937,120.289958,DST
14399,82.739488,DTNA
...,...,...
2553,-68.445317,NTRK2
19091,-81.739050,GPM6A
6496,-86.092071,NTM
